In [2]:
import pandas as pd
from Bio import SeqIO
import os

In [3]:
file_path = "files/sprot_human.fasta"

In [4]:
sequences = [i for i in SeqIO.parse(file_path,'fasta')]

In [8]:
len(sequences)

20341

In [12]:
sequence_one = sequences[20]
sequence_one.name

'sp|Q8N2K0|ABD12_HUMAN'

In [23]:
first_record = sequence[0]
f_id = first_record.id
f_name = first_record.name
f_description = first_record.description

print(f_id)
print(f_name)
print(f_description)

sp|P62258|1433E_HUMAN
sp|P62258|1433E_HUMAN
sp|P62258|1433E_HUMAN 14-3-3 protein epsilon OS=Homo sapiens OX=9606 GN=YWHAE PE=1 SV=1


In [24]:
first_sequence = first_record.seq
print(len(first_sequence))

255
